### helper notebook to show how to generate maps based off of preds.csv after running the pipeline

authored nov 14, 2025 by jelshawa

#### context
to run the pipeline and generate predictions starting from the latest noaa recorded time, execute this command
> !python main.py --config config.yaml

In [1]:
%%time
!python main.py --config config.yaml

2025-11-17 09:21:11,606 — INFO — 🔥 starting inference pipeline!
2025-11-17 09:21:11,606 — INFO — 📂 run dir: ./TESTS/MAIN_TESTRUN2
2025-11-17 09:21:11,606 — INFO — 📦 loading pretrained artifacts...
2025-11-17 09:21:11,763 — INFO — 🧠 loading trained model weights...
2025-11-17 09:21:11,782 — INFO — 🔧 using patched model_args: {'in_channels': 8, 'out_channels': 1, 'hidden_dim': 32, 'pred_horizons': 7, 'num_nodes': 14005}
2025-11-17 09:21:11,837 — INFO — loaded model GCNGRU with args: {'in_channels': 8, 'out_channels': 1, 'hidden_dim': 32, 'pred_horizons': 7, 'num_nodes': 14005}
2025-11-17 09:21:11,838 — INFO — in main extraction func, hours: 24, lat min: 42.0
2025-11-17 09:21:12,407 — INFO — 🌟 latest available urma timestamp: 2025-11-17 07:00:00
2025-11-17 09:22:14,047 — INFO — [⏱️ Load NOAA data] took 62.21 sec
2025-11-17 09:22:14,047 — INFO — 
====================== ✅ extraction successful ======================
2025-11-17 09:22:14,047 — INFO — 🧪 loaded df with shape: (336120, 9)
2025-1

#### imports + helper functions

In [2]:
import yaml
from argparse import Namespace
import pandas as pd
import numpy as np
import torch
from util.plotting import plot_inference_maps, plot_prediction_timeseries
from datetime import timedelta
import os

In [3]:
def load_config(config_path):
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)

    def to_namespace(obj):
        if isinstance(obj, dict):
            ns = Namespace()
            for k, v in obj.items():
                setattr(ns, k, to_namespace(v))
            return ns
        return obj

    return to_namespace(cfg)

#### main setup

In [4]:
args = load_config("./config.yaml")

In [5]:
nodes_csv_path = args.graph.nodes_csv_path

In [6]:
# load preds csv
pred_csv_path = f"{args.run_dir}/predictions.csv"
df_pred = pd.read_csv(pred_csv_path)

print("loaded preds csv:")
df_pred.head()

loaded preds csv:


,forecast_time,horizon_hours,lat,lon,predicted_temp
0,2025-11-17 08:00:00,1,42.000080,-80.84195,5.121700
1,2025-11-17 08:00:00,1,42.000153,-78.74701,-3.153623
2,2025-11-17 08:00:00,1,42.000496,-79.24889,-1.928824
3,2025-11-17 08:00:00,1,42.000782,-78.51062,-2.723086
4,2025-11-17 08:00:00,1,42.001110,-78.27423,-0.600414


In [7]:
# get num of horizons from csv
pred_offsets = sorted(df_pred["horizon_hours"].unique().tolist())
H = len(pred_offsets)

# get num of nodes
unique_nodes = df_pred[["lat", "lon"]].drop_duplicates()
N = len(unique_nodes)

print(f"horizons (H): {H}")
print(f"nodes (N): {N}")

horizons (H): 7
nodes (N): 14005


In [8]:
pred_tensor = np.zeros((1, H, N, 1), dtype=np.float32)

# sort df by horizon + (lat, lon)
df_sorted = df_pred.sort_values(["horizon_hours", "lat", "lon"]).reset_index(drop=True)

for h_idx, horizon in enumerate(pred_offsets):
    df_h = df_sorted[df_sorted["horizon_hours"] == horizon]
    pred_tensor[0, h_idx, :, 0] = df_h["predicted_temp"].values

In [9]:
# double check shape, should be (batch_size, # horizons, # nodes, output dim)
preds_unscaled_from_csv = pred_tensor
preds_unscaled_from_csv.shape 
# so for current setup should be (1, 7, 14005, 1) 
# bc only doing one 'sequence', for 7 horizons, have 14005 nodes, and 1 pred for each node per horizon (temp)

(1, 7, 14005, 1)

In [10]:
# double check that num nodes match our nodes df
nodes_df = pd.read_csv(nodes_csv_path)
nodes_df = nodes_df.sort_values(["lat", "lon"]).reset_index(drop=True)
assert len(nodes_df) == N

In [11]:
# get latest timestamp that we used for generating the predictions
h_min = int(df_pred["horizon_hours"].min()) # min of the horizons
t_forecast_0 = pd.to_datetime(df_pred["forecast_time"]).min() # min of the timestamps

latest_timestamp = t_forecast_0 - timedelta(hours=h_min) # latest ts = min of the timestamps - min horizon time

print("recovered latest_timestamp:", latest_timestamp)

recovered latest_timestamp: 2025-11-17 07:00:00


#### plotting

In [12]:
# create plot dir
plot_dir = f"{args.run_dir}/plots_from_csv"
os.makedirs(plot_dir, exist_ok=True)

# plot spatial maps
print("generating spatial inference maps from csv...")
plot_inference_maps(
    preds_unscaled=preds_unscaled_from_csv,
    nodes_df=nodes_df,
    pred_offsets=pred_offsets,
    out_dir=os.path.join(plot_dir, "maps"),
    latest_timestamp=latest_timestamp
)

generating spatial inference maps from csv...
2025-11-17 09:22:29,121 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-17_08-00_plus1h.png
2025-11-17 09:22:29,582 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-17_13-00_plus6h.png
2025-11-17 09:22:30,039 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-17_19-00_plus12h.png
2025-11-17 09:22:30,496 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-18_01-00_plus18h.png
2025-11-17 09:22:30,955 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-18_07-00_plus24h.png
2025-11-17 09:22:31,416 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-18_19-00_plus36h.png
2025-11-17 09:22:31,868 — INFO — 📌 saved spatial map → ./TESTS/MAIN_TESTRUN2/plots_from_csv/maps/spatial_2025-11-19_07-00_plus48h.png


In [13]:
# plot timeseries
print("generating timeseries plots from csv...")
plot_prediction_timeseries(
    preds_unscaled=preds_unscaled_from_csv,
    pred_offsets=pred_offsets,
    out_dir=os.path.join(plot_dir, "timeseries"),
    latest_timestamp=latest_timestamp
)

generating timeseries plots from csv...
2025-11-17 09:22:32,079 — INFO — 📌 saved mean-timeseries → ./TESTS/MAIN_TESTRUN2/plots_from_csv/timeseries/mean_timeseries_20251117_0700.png
2025-11-17 09:22:32,280 — INFO — 📌 saved node-timeseries → ./TESTS/MAIN_TESTRUN2/plots_from_csv/timeseries/node_1084_timeseries_20251117_0700.png
2025-11-17 09:22:32,633 — INFO — 📌 saved node-timeseries → ./TESTS/MAIN_TESTRUN2/plots_from_csv/timeseries/node_2854_timeseries_20251117_0700.png
2025-11-17 09:22:32,861 — INFO — 📌 saved node-timeseries → ./TESTS/MAIN_TESTRUN2/plots_from_csv/timeseries/node_4550_timeseries_20251117_0700.png
2025-11-17 09:22:33,066 — INFO — 📌 saved node-timeseries → ./TESTS/MAIN_TESTRUN2/plots_from_csv/timeseries/node_5089_timeseries_20251117_0700.png
2025-11-17 09:22:33,270 — INFO — 📌 saved node-timeseries → ./TESTS/MAIN_TESTRUN2/plots_from_csv/timeseries/node_6719_timeseries_20251117_0700.png
